<a href="https://colab.research.google.com/github/rcofrer/Coursera_Capstone/blob/master/Clustering_Toronto_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Scrapping Wikipedia Table

In [0]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
soup = BeautifulSoup(results.text, 'html.parser')

data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 

## Creating DataFrame

In [141]:
df = pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


## Cleaning the DataFrame

In [142]:
df  = df[~(df['Borough']=='Not assigned')]
df  = df.loc[1:,:]
df.reset_index(drop=True,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## DataFrame shape

In [143]:
df.shape

(103, 3)

------------------------------------------------------------

---------------------------------

# Next task

# Adding Latitude and Longitude

In [0]:
import geocoder # import geocoder

# function to get the latitude and longitude

def get_lat_long(postal_code):
  lat_lng_coords = None
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    try:
      latitude = lat_lng_coords[0]
      longitude = lat_lng_coords[1]
    except:
      pass
    print('trying') 
  return([latitude,longitude])


## USING CSV. THIS TAKES TOO MUCH TIME

## Reading csv file with the information

In [0]:
lat_long_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [145]:
lat_long_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
lat_long_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging both DataFrames

In [147]:
df_Toronto = pd.merge(df,lat_long_df,on='PostalCode')
df_Toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
